<a href="https://colab.research.google.com/github/HabibKedir/HabibKedir/blob/main/AmhEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow transformers

In [2]:
from google.colab import drive
import numpy as np  # Import numpy
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set directory paths
data_dir = "/content/drive/MyDrive/New"
amh_path = os.path.join(data_dir, "Amh.txt")
eng_path = os.path.join(data_dir, "Eng.txt")

# Load data
with open(amh_path, 'r', encoding='utf-8') as f:
    amh_sentences = f.readlines()

with open(eng_path, 'r', encoding='utf-8') as f:
    eng_sentences = f.readlines()

print(f"Loaded {len(amh_sentences)} Amharic sentences and {len(eng_sentences)} English sentences.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 10892 Amharic sentences and 10892 English sentences.


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
def create_tokenizer(sentences):
    tokenizer = Tokenizer(filters='', oov_token="<unk>")
    tokenizer.fit_on_texts(sentences)
    return tokenizer

amh_tokenizer = create_tokenizer(amh_sentences)
eng_tokenizer = create_tokenizer(eng_sentences)

amh_vocab_size = len(amh_tokenizer.word_index) + 1
eng_vocab_size = len(eng_tokenizer.word_index) + 1

# Preprocess sentences
def preprocess_sentences(sentences, tokenizer, max_length=30):
    sequences = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_length, padding='post')

max_length = 30
amh_data = preprocess_sentences(amh_sentences, amh_tokenizer, max_length)
eng_data = preprocess_sentences(eng_sentences, eng_tokenizer, max_length)

print(f"Amharic data shape: {amh_data.shape}, English data shape: {eng_data.shape}")


Amharic data shape: (10892, 30), English data shape: (10892, 30)


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Define the Seq2Seq Model
class Seq2SeqModel(tf.keras.Model):
    def __init__(self, encoder_vocab_size, decoder_vocab_size, embedding_dim, units):
        super(Seq2SeqModel, self).__init__()
        self.encoder_embedding = Embedding(encoder_vocab_size, embedding_dim)
        self.encoder_lstm = LSTM(units, return_state=True)

        self.decoder_embedding = Embedding(decoder_vocab_size, embedding_dim)
        self.decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
        self.fc = Dense(decoder_vocab_size)

    def call(self, inputs, training=False):
        enc_input, dec_input = inputs

        # Encoder
        enc_embedded = self.encoder_embedding(enc_input)
        _, enc_h, enc_c = self.encoder_lstm(enc_embedded)

        # Decoder
        dec_embedded = self.decoder_embedding(dec_input)
        dec_output, _, _ = self.decoder_lstm(dec_embedded, initial_state=[enc_h, enc_c])
        output = self.fc(dec_output)

        return output


In [5]:
!pip install nltk scikit-learn
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from sklearn.model_selection import KFold
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

# BLEU metric (unchanged)
def calculate_bleu(predictions, references):
    references = [[ref] for ref in references]
    return corpus_bleu(references, predictions)

# K-Fold Split (moved here to be before its usage)
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

embedding_dim = 256
units = 512
batch_size = 16
epochs = 5
bleu_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(amh_data)):
    print(f"Training on Fold {fold + 1}/{num_folds}...")

    # Split data
    train_amh, val_amh = amh_data[train_idx], amh_data[val_idx]
    train_eng, val_eng = eng_data[train_idx], eng_data[val_idx]

    # Define the model
    model = Seq2SeqModel(amh_vocab_size, eng_vocab_size, embedding_dim, units)

    # Compile the model
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=loss_object,
                  metrics=['accuracy'])

    # Train the model
    model.fit(
        [train_amh, train_eng[:, :-1]], train_eng[:, 1:],
        validation_data=([val_amh, val_eng[:, :-1]], val_eng[:, 1:]),
        batch_size=batch_size,
        epochs=epochs
    )

    # Evaluate on validation set
    val_predictions = model.predict([val_amh, val_eng[:, :-1]])
    val_predictions = tf.argmax(val_predictions, axis=-1).numpy()

    bleu = calculate_bleu(val_predictions, val_eng[:, 1:])
    bleu_scores.append(bleu)
    print(f"BLEU score for Fold {fold + 1}: {bleu:.4f}")

# Average BLEU score
print(f"Average BLEU score across folds: {np.mean(bleu_scores):.4f}")


Training on Fold 1/5...
Epoch 1/5
545/545 [==============================] - 167s 300ms/step - loss: 5.1909 - accuracy: 0.3711 - val_loss: 4.7871 - val_accuracy: 0.3920
Epoch 2/5
545/545 [==============================] - 162s 297ms/step - loss: 4.5000 - accuracy: 0.4048 - val_loss: 4.5641 - val_accuracy: 0.4147
Epoch 3/5
545/545 [==============================] - 162s 298ms/step - loss: 4.1290 - accuracy: 0.4297 - val_loss: 4.4702 - val_accuracy: 0.4251
Epoch 4/5
545/545 [==============================] - 163s 299ms/step - loss: 3.7887 - accuracy: 0.4462 - val_loss: 4.4477 - val_accuracy: 0.4325
Epoch 5/5
69/69 [==============================] - 7s 94ms/step
BLEU score for Fold 1: 0.3494
Training on Fold 2/5...
Epoch 1/5
545/545 [==============================] - 170s 306ms/step - loss: 5.1856 - accuracy: 0.3723 - val_loss: 4.8086 - val_accuracy: 0.3927
Epoch 2/5
545/545 [==============================] - 167s 306ms/step - loss: 4.5106 - accuracy: 0.4031 - val_loss: 4.6028 - val_accur